In [1]:
import os
import numpy as np 
import pandas as pd
from glob import glob
import cv2
import random
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import load_model

In [2]:
test_list = pd.read_csv(r'..\Paper\test_path.csv').iloc[:,1:]
test_path = '..\Paper\Test\\'
test_list['File Paths'] = test_list['File Paths'].apply(lambda path:os.path.join(test_path, path))
df_drop_test = test_list.drop_duplicates('code',keep='first').reset_index(drop=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1204814147.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_list = pd.read_csv(r'..\Paper\test_path.csv').iloc[:,1:]


In [3]:
model = load_model(r'..\Paper\model_V2.h5')

In [5]:
backtest = pd.DataFrame()
lower = 1750
upper = 1800
for code in df_drop_test['code'].values[lower:upper]:
    test_df = test_list[test_list['code'] == code].reset_index(drop=True)

    img_list_test = []
    for i in range(len(test_df)):
        img_test = cv2.imread(test_df['File Paths'].loc[i])
        img_test = cv2.resize(img_test, (323, 172))
        img_list_test.append(img_test)
    X_test = np.concatenate(img_list_test, axis = 0)

    X_test = X_test.reshape(len(test_df), 323,172,3)/255
    y_prob = model.predict(X_test)
    y_prob_flat = np.ravel(y_prob)
    y_prob_df = pd.DataFrame({'predict_rate':y_prob_flat})
    y_prob_df['Decision'] = y_prob_df['predict_rate'].apply(lambda value:'Sell' if value<=y_prob_df['predict_rate'].min() else ('Buy' if value >= y_prob_df['predict_rate'].max() else 'No Action'))

    backtest_df = pd.concat([test_df['code'],test_df['Date'],y_prob_df['Decision'],y_prob_df['predict_rate'],test_df['Close']],axis=1)
    backtest_df['RR'] = 0
    buy_index = backtest_df[backtest_df['Decision'] == 'Buy'].index[0]
    sell_index = backtest_df[backtest_df['Decision'] == 'Sell'].index[0]

    if buy_index > sell_index:
        backtest_df.loc[sell_index, 'RR'] = backtest_df.loc[sell_index, 'Close'] /backtest_df.loc[buy_index, 'Close'] - 1
    elif buy_index < sell_index:
        backtest_df.loc[buy_index, 'RR']  = backtest_df.loc[sell_index, 'Close']/backtest_df.loc[buy_index, 'Close'] - 1
    backtest = pd.concat([backtest,backtest_df])


C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.05882352941176472' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[sell_index, 'RR'] = backtest_df.loc[sell_index, 'Close'] /backtest_df.loc[buy_index, 'Close'] - 1
C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.037871033776867846' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[buy_index, 'RR']  = backtest_df.loc[sell_index, 'Close']/backtest_df.loc[buy_index, 'Close'] - 1
C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Va

C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.18760862354892205' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[sell_index, 'RR'] = backtest_df.loc[sell_index, 'Close'] /backtest_df.loc[buy_index, 'Close'] - 1
C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.041361667057752904' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[buy_index, 'RR']  = backtest_df.loc[sell_index, 'Close']/backtest_df.loc[buy_index, 'Close'] - 1
C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Va

C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.17934782608695643' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[buy_index, 'RR']  = backtest_df.loc[sell_index, 'Close']/backtest_df.loc[buy_index, 'Close'] - 1
C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.10778423772609824' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[sell_index, 'RR'] = backtest_df.loc[sell_index, 'Close'] /backtest_df.loc[buy_index, 'Close'] - 1
C:\Users\USER\AppData\Local\Temp\ipykernel_15412\1890274416.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Val

"    \n    RR_5d = []\n    RR_10d = []\n    RR_15d = []\n    RR_20d = []\n    RR_30d = []\n    RR_60d = []\n    RR_90d = []\n    RR_120d = []\n    RR_180d = []\n    RR_240d = []\n\n    for i in range(len(test_df)):\n\n        if backtest_df['Decision'].loc[i] == 'Buy' and i+5 < len(test_df):\n            RR = (backtest_df['Close'].loc[i+5] / backtest_df['Close'].loc[i]) - 1\n        elif backtest_df['Decision'].loc[i] == 'Sell' and i+5 < len(test_df):\n            RR = (backtest_df['Close'].loc[i] / backtest_df['Close'].loc[i+5]) - 1 \n        else:\n            RR = 0\n        RR_5d.append(RR)\n        \n        if backtest_df['Decision'].loc[i] == 'Buy' and i+10 < len(test_df):\n            RR = (backtest_df['Close'].loc[i+10] / backtest_df['Close'].loc[i]) - 1\n        elif backtest_df['Decision'].loc[i] == 'Sell' and i+10 < len(test_df):\n            RR = (backtest_df['Close'].loc[i] / backtest_df['Close'].loc[i+10]) - 1 \n        else:\n            RR = 0\n        RR_10d.append(RR

In [6]:
backtest.to_csv('backtest_' + str(lower) +'_' + str(upper) + '.csv')